# SICONFI notebook - Consulta ao Serviço de Dados Abertos do Sistema de Informações Contábeis e Fiscais do Setor Público Brasileiro

## Sobre

O SICONFI notebook é um código desenvolvido em linguagem Python para acesso a dados do SICONFI via API do serviço de dados abertos da [Secretaria do Tesouro Nacional](https://www.gov.br/tesouronacional/).

* Versão: 10.10.2022
* Licenciamento: [GNU General Public License v3.0](https://www.gnu.org/licenses/gpl-3.0.html)

## Leia-me

A aplicação otimiza a coleta de grandes volumes de dados do SICONFI por meio de uma série de consultas recursivas. A aplicação realiza essa coleta de duas formas:

* Por Estados, ao informar-se a sigla de um ou mais Estados brasileiros
* Por Estado e seus Municípios, ao informar-se a sigla de um único Estado brasileiro

Para seu adequado funcionamento, faz-se necessária a instalação das seguintes bibliotecas da linguagem Python 3:

* json
* requests
* pandas
* openpyxl
* time
* timeit

## Referências

* [API do SICONFI](http://apidatalake.tesouro.gov.br/docs/siconfi/#/)
* [Manual de Demonstrativos Fiscais (MDF)](https://www.gov.br/tesouronacional/pt-br/contabilidade-e-custos/manuais/manual-de-demonstrativos-fiscais-mdf) da [Secretaria do Tesouro Nacional](https://www.gov.br/tesouronacional/)

# Bibliotecas, Funções e Classes

In [ ]:
!pip install --upgrade pip
!pip install requests
!pip install pandas
!pip install openpyxl
import json, requests, time, timeit, pandas as pd

## Conexão e Carga de Dados

In [ ]:
def ConectarAPI(host, dataset, filters):
    """Função que estabelece uma conexão via API REST"""
    url = host + dataset + filters
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.HTTPError:
        print(msg_conexao.format(response.url, response.status_code, response.reason))
        raise

def CarregarDados(arquivo, base):
    """Função que coleta dados no formato JSON e os armazena num DataFrame"""
    dados = json.loads(arquivo)
    if base == 'siconfi':
        df = pd.DataFrame(dados.get('items'))
    elif base == 'ibge':
        df = pd.DataFrame(dados)
    return df

## Definição de Parâmetros

In [ ]:
def BuscarEstado(host, dataset, filters, uf):
    """Função para buscar um Estado na base de dados do IBGE"""
    df = CarregarDados(ConectarAPI(host, dataset, filters), 'ibge')
    filtro = df.loc[df['UF-sigla'] == uf]
    if not filtro.empty:
        estado = list(filtro['UF-id'])
        return estado[0]
    else:
        return 0

def ListarMunicipios(host, dataset, filters, uf):
    """Função que retorna uma lista de códigos de Municípios"""
    dataset = dataset[:20] + uf + dataset[24:]
    df = CarregarDados(ConectarAPI(host, dataset, filters), 'ibge')
    munic = list(df['municipio-id'])
    return munic

def ListarEntes(ente, entes, tipo):
    """Função que retorna uma lista de códigos de entes"""
    estado = BuscarEstado(host_ibge, dataset_ibge_estados, filters_ibge_estados, ente)
    if estado != 0:
        entes.append(estado)
        if tipo == '2':
            entes.extend(ListarMunicipios(host_ibge, dataset_ibge_munic, filters_ibge_munic, ente))
    return entes

def DefinirEntes(tipo):
    """Função para definir o parâmetro 'id_ente'"""
    ExibirMenuVertical(menu_estado)
    entes = list()
    while True:
        ente = str.upper(input(msg_sigla_estado))
        if ente.isnumeric() == False and len(ente) == 2:
            entes = ListarEntes(ente, entes, tipo)
            if tipo == '2':
                break
        else:
            break
    return entes

def ListarPoderes(poder, poderes):
    """
    Função que retorna uma lista de códigos de poderes
    
    Código do poder: T = Todos os poderes, 1 = Legislativo, 2 = Executivo, 3 = Judiciário, 4 = Ministério Público e 5 = Defensoria Pública
    """
    dict_poderes = {'1': 'L', '2': 'E', '3': 'J', '4': 'M', '5': 'D'}
    if poder in dict_poderes.keys():
        if dict_poderes[poder] not in poderes:
            poderes.append(dict_poderes[poder])
    else:
        poderes = list(dict_poderes.values())
    return poderes

def DefinirPoderes():
    """Função para definir o parâmetro 'co_poder'"""
    ExibirMenuHorizontal(menu_poder)
    poderes = list()
    while True:
        poder = str.upper(input(msg_codigo_poder))
        if (poder.isnumeric() and int(poder) in range(1,5)) or (poder == 'T'):
            poderes = ListarPoderes(poder, poderes)
            if poder == 'T':
                break
        else:
            break
    return poderes

def ListarExercicios(exercicio_inicio, exercicio_fim):
    """Função que retorna uma lista de exercícios"""
    exercicios = [str(exercicio) for exercicio in range (exercicio_inicio, exercicio_fim + 1)]
    return exercicios

def DefinirExercicios():
    """
    Função para definir o parâmetro 'an_exercicio'

    Exercício do relatório: a partir de 2015
    """
    ExibirMenuVertical(menu_exercicio)
    while True:
        exercicio_inicio = str.upper(input(msg_exercicio_inicio))
        exercicio_fim = str.upper(input(msg_exercicio_fim))
        if exercicio_inicio.isnumeric() and exercicio_fim.isnumeric() and int(exercicio_inicio) in range(2015,2065,1) and int(exercicio_fim) in range(2015,2065,1):
            return ListarExercicios(int(exercicio_inicio), int(exercicio_fim))

def ListarPeriodos(periodo, periodos, periodicidade):
    """
    Função que retorna uma lista de períodos conforme a periodicidade de publicação de um relatório
    
    Período de referência do relatório: T, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13
    
    Periodicidade de publicação do relatório: M = Mensal, B = Bimestral, Q = Quadrimestral e S = Semestral
    """
    lista_periodos = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
    if periodo != 'T':
        if periodicidade == 'M' or periodicidade == 'B' or periodicidade == 'Q' or periodicidade == 'S':
            periodos.append(periodo)
        else:
            periodos = lista_periodos
    elif periodicidade == 'M':
        periodos = lista_periodos
    elif periodicidade == 'B':
        periodos = lista_periodos[:6]
    elif periodicidade == 'Q':
        periodos = lista_periodos[:3]
    elif periodicidade == 'S':
        periodos = lista_periodos[:2]
    return periodos

def DefinirPeriodos(menu, periodicidade):
    """Função para definir o parâmetro 'nr_periodo' conforme a periodicidade de publicação de um relatório"""
    ExibirMenuHorizontal(menu)
    periodos = list()
    while True:
        periodo = str.upper(input(msg_periodo))
        if (periodo.isnumeric() and int(periodo) in range(1,14)) or (periodo == 'T'):
            periodos = ListarPeriodos(periodo, periodos, periodicidade)
            if periodo == 'T':
                break
        else:
            break
    return periodos

def ListarAnexosRREO(anexo, anexos):
    """
    Função que retorna uma lista de códigos de anexos do Relatório Resumido da Execução Orçamentária - RREO
    
    Código do anexo: T = Todos os anexos, 1 = Balanço Orçamentário, 2 = Despesas por Função e Subfunção, 3 = Receita Corrente Líquida,
    
    4 = Receitas e Despesas Previdenciárias, 4G = Anexo 04 - RGPS, 4P = Anexo 04 - RPPS, 4.0 = Anexo 04.0 - RGPS, 4.1 = Anexo 04.1, 4.2 = Anexo 04.2, 4.3 = Anexo 04.3 - RGPS,
    
    5 = RREO-Anexo 05, 6 = Resultados Primário e Nominal, 7 = Restos a Pagar por Poder e Órgão, 9 = Receitas de Operações de Crédito e Despesas de Capital,
    
    10G = Projeção Atuarial RGPS, 10P = Projeção Atuarial RPPS, 11 = Receita de Alienação de Ativos e Aplicação dos Recursos,
    
    13 = Parcerias Público-Privadas, 14 = RREO Simplificado
    """
    dict_anexos = {'1': 'RREO-Anexo 01', '2': 'RREO-Anexo 02', '3': 'RREO-Anexo 03', '4': 'RREO-Anexo 04', '4G': 'RREO-Anexo 04 - RGPS', '4P': 'RREO-Anexo 04 - RPPS', '4.0': 'RREO-Anexo 04.0 - RGPS', '4.1': 'RREO-Anexo 04.1', '4.2': 'RREO-Anexo 04.2', '4.3': 'RREO-Anexo 04.3 - RGPS', '5': 'RREO-Anexo 05', '6': 'RREO-Anexo 06', '7': 'RREO-Anexo 07', '9': 'RREO-Anexo 09', '10G': 'RREO-Anexo 10 - RGPS', '10P':'10.2 RREO-Anexo 10 - RPPS', '11': 'RREO-Anexo 11', '13': 'RREO-Anexo 13', '14': 'RREO-Anexo 14'}
    if anexo in dict_anexos.keys():
        if dict_anexos[anexo] not in anexos:
            anexos.append(dict_anexos[anexo])
    else:
        anexos = list(dict_anexos.values())
    return anexos

def DefinirAnexoRREO():
    """Função para definir o parâmetro 'no_anexo' do Relatório Resumido da Execução Orçamentária - RREO"""
    ExibirMenuVertical(menu_rreo_anexo)
    anexos = list()
    while True:
        anexo = str.upper(input(msg_rreo_anexo))
        if (anexo.isnumeric() and int(anexo) in range(1,15)) or (anexo == 'T'):
            anexos = ListarAnexosRREO(anexo, anexos)
            if anexo == 'T':
                break
        else:
            break
    return anexos

def ListarAnexosRGF(anexo, anexos):
    """
    Função que retorna uma lista de códigos de anexos do Relatório da Gestão Fiscal - RGF
    
    Código do anexo: T = Todos os anexos, 1 = Despesa com Pessoal, 2 = Dívida Consolidada Líquida, 3 = Garantias e Contragarantias de Valores,
    
    4 = Operações de Crédito, 5 = Disponibilidade de Caixa e dos Restos a Pagar e 6 = RGF Simplificado
    """
    dict_anexos = {'1': 'RGF-Anexo 01', '2': 'RGF-Anexo 02', '3': 'RGF-Anexo 03', '4': 'RGF-Anexo 04', '5': 'RGF-Anexo 05', '6': 'RGF-Anexo 06'}
    if anexo in dict_anexos.keys():
        if dict_anexos[anexo] not in anexos:
            anexos.append(dict_anexos[anexo])
    else:
        anexos = list(dict_anexos.values())
    return anexos

def DefinirAnexoRGF():
    """Função para definir o parâmetro 'no_anexo' do Relatório da Gestão Fiscal - RGF"""
    ExibirMenuVertical(menu_rgf_anexo)
    anexos = list()
    while True:
        anexo = str.upper(input(msg_rgf_anexo))
        if (anexo.isnumeric() and int(anexo) in range(1,7)) or (anexo == 'T'):
            anexos = ListarAnexosRGF(anexo, anexos)
            if anexo == 'T':
                break
        else:
            break
    return anexos

def ListarAnexosDCA(anexo, anexos):
    """
    Função que retorna uma lista de códigos de anexos da Declaração de Contas Anuais - DCA
    
    Código do anexo: T = Todos os anexos, 1 = Balanço Patrimonial, 2 = Receitas Orçamentárias, 3 = Despesas Orçamentárias,
    
    4 = Despesas por Função, 5 = Execução de Restos a Pagar, 6 = Execução de Restos a Pagar por Função, 7 = Variações Patrimoniais
    """
    dict_anexos = {'1': 'DCA-Anexo I-AB', '2': 'DCA-Anexo I-C', '3': 'DCA-Anexo I-D', '4': 'DCA-Anexo I-E', '5': 'DCA-Anexo I-F', '6': 'DCA-Anexo I-G', '7': 'DCA-Anexo I-HI'}
    if anexo in dict_anexos.keys():
        if dict_anexos[anexo] not in anexos:
            anexos.append(dict_anexos[anexo])
    else:
        anexos = list(dict_anexos.values())
    return anexos

def DefinirAnexoDCA():
    """Função para definir o parâmetro 'no_anexo' da Declaração de Contas Anuais - DCA"""
    ExibirMenuVertical(menu_dca_anexo)
    anexos = list()
    anexos_aux = list()
    while True:
        anexo = str.upper(input(msg_dca_anexo))
        if (anexo.isnumeric() and int(anexo) in range(1,8)) or (anexo == 'T'):
            anexos = ListarAnexosDCA(anexo, anexos)
            if anexo == 'T':
                break
        else:
            break
    return anexos

def ListarTipoMatriz(tipo, tipos):
    """
    Função que retorna uma lista de códigos de tipos de Matriz de Saldos Contábeis - MSC
    
    Código do tipo de matriz: T = Todos os tipos, MSCC = Matriz Agregada, MSCE = Matriz Encerramento
    """
    dict_tipos = {'1': 'MSCC', '2': 'MSCE'}
    if tipo in dict_tipos.keys():
        if dict_tipos[tipo] not in tipos:
            tipos.append(dict_tipos[tipo])
    else:
        tipos = list(dict_tipos.values())
    return tipos

def DefinirTipoMatriz():
    """Função para definir o parâmetro 'co_tipo_matriz' da Matriz de Saldos Contábeis - MSC"""
    ExibirMenuHorizontal(menu_msc_tipo_matriz)
    tipos = list()
    while True:
        tipo = str.upper(input(msg_msc_tipo_matriz))
        if (tipo.isnumeric() and int(tipo) in range(1,3)) or (tipo == 'T'):
            tipos = ListarTipoMatriz(tipo, tipos)
            if tipo == 'T':
                break
        else:
            break
    return tipos

def ListarClasseConta(classe, classes, dataset):
    """
    Função que retorna uma lista de códigos de classes de contas da Matriz de Saldos Contábeis - MSC
    
    Código da classe de conta: 1, 2, 3, 4, 5, 6, 7, 8
    """
    lista_classes = ['1', '2', '3', '4', '5', '6', '7', '8']
    if dataset == 'msc_patrimonial':
        if classe in lista_classes[:4]:
            if classe not in classes:
                classes.append(classe)
        else:
            classes = lista_classes[:4]
    elif dataset == 'msc_orcamentaria':
        if classe in lista_classes[4:6]:
            if classe not in classes:
                classes.append(classe)
        else:
            classes = lista_classes[4:6]
    elif dataset == 'msc_controle':
        if classe in lista_classes[6:8]:
            if classe not in classes:
                classes.append(classe)
        else:
            classes = lista_classes[6:8]
    return classes

def DefinirClasseConta(dataset):
    """Função para definir o parâmetro 'classe_conta' da Matriz de Saldos Contábeis - MSC"""
    if dataset == 'msc_patrimonial':
        ExibirMenuHorizontal(menu_msc_classe_conta_patrimonial)
    elif dataset == 'msc_orcamentaria':
        ExibirMenuHorizontal(menu_msc_classe_conta_orcamentaria)
    elif dataset == 'msc_controle':
        ExibirMenuHorizontal(menu_msc_classe_conta_controle)
    classes = list()
    while True:
        classe = str.upper(input(msg_msc_classe_conta))
        if (classe.isnumeric() and int(classe) in range(1,9)) or (classe == 'T'):
            classes = ListarClasseConta(classe, classes, dataset)
            if classe == 'T':
                break
        else:
            break
    return classes

def ListarTipoValor(tipo, tipos):
    """
    Função que retorna uma lista de códigos de tipos de valor da Matriz de Saldos Contábeis - MSC
    
    Código do tipo de valor: T = Todos os tipos, beginning_balance = Saldo inicial, period_change = Movimento, ending_balance = Saldo final
    """
    dict_tipos = {'1': 'beginning_balance', '2': 'period_change', '3': 'ending_balance'}
    if tipo in dict_tipos.keys():
        if dict_tipos[tipo] not in tipos:
            tipos.append(dict_tipos[tipo])
    else:
        tipos = list(dict_tipos.values())
    return tipos

def DefinirTipoValor():
    """Função para definir o parâmetro 'id_tv' da MSC"""
    ExibirMenuHorizontal(menu_msc_tipo_valor)
    tipos = list()
    while True:
        tipo = str.upper(input(msg_msc_tipo_valor))
        if (tipo.isnumeric() and int(tipo) in range(1,4)) or (tipo == 'T'):
            tipos = ListarTipoValor(tipo, tipos)
            if tipo == 'T':
                break
        else:
            break
    return tipos
    
def DefinirDataset(n):
    """Função para definir o conjunto de dados e o nome do arquivo que será gerado"""
    datasets = {'1': 'anexos-relatorios', '2': 'entes', '3': 'extrato_entregas', '4': 'rreo', '5': 'rgf', '6': 'dca', '7': 'msc_patrimonial', '8': 'msc_orcamentaria', '9': 'msc_controle'}
    arquivos = {'1': 'SICONFI_Anexos.xlsx', '2': 'SICONFI_Entes.xlsx', '3': 'SICONFI_Entregas.xlsx', '4': 'SICONFI_RREO.xlsx', '5': 'SICONFI_RGF.xlsx', '6': 'SICONFI_DCA.xlsx', '7': 'SICONFI_MSC_Patrimonial.xlsx', '8': 'SICONFI_MSC_Orcamentaria.xlsx', '9': 'SICONFI_MSC_Controle.xlsx'}
    return (datasets[n], arquivos[n]) if n in datasets.keys() else (0,0)

## Processamento de Conjuntos de Dados

In [ ]:
def ProcessarEntregas():
    """Função que processa o dataset extrato_entregas"""
    df = pd.DataFrame()
    for an_exercicio in exercicios:
        for id_ente in entes:
            df_aux = pd.DataFrame()
            filters_siconfi = '?id_ente=' + str(id_ente) + '&an_referencia=' + an_exercicio
            time.sleep(tempo)
            df_aux = CarregarDados(ConectarAPI(host_siconfi, dataset_siconfi[0], filters_siconfi), 'siconfi')
            if df.empty:
                df = df_aux
            else:
                df = pd.concat([df, df_aux])
    return df

def ProcessarRREO():
    """Função que processa o dataset rreo"""
    n = 0
    ciclo = len(exercicios) * len(periodos)
    df = pd.DataFrame()
    for an_exercicio in exercicios:
        for nr_periodo in periodos:
            for id_ente in entes:
                for no_anexo in anexos:
                    df_aux = pd.DataFrame()
                    if len(str(id_ente)) == 2:
                        co_esfera = 'E'
                    else:
                        co_esfera = 'M'
                    filters_siconfi = '?an_exercicio=' + an_exercicio + '&nr_periodo=' + nr_periodo + '&co_tipo_demonstrativo=' + tipo_demonstrativo + '&no_anexo=' + no_anexo + '&co_esfera=' + co_esfera + '&id_ente=' + str(id_ente)
                    time.sleep(tempo)
                    df_aux = CarregarDados(ConectarAPI(host_siconfi, dataset_siconfi[0], filters_siconfi), 'siconfi')
                    if df.empty:
                        df = df_aux
                    else:
                        df = pd.concat([df, df_aux], ignore_index=True)
            n += 1
            progresso = n/ciclo*100
            print(msg_processamento_progresso.format(progresso))
    return df

def ProcessarRGF():
    """Função que processa o dataset rgf"""
    n = 0
    ciclo = len(exercicios) * len(periodos)
    df = pd.DataFrame()
    for an_exercicio in exercicios:
        for nr_periodo in periodos:
            for id_ente in entes:
                for no_anexo in anexos:
                    for co_poder in poderes:
                        df_aux = pd.DataFrame()
                        if len(str(id_ente)) == 2:
                            co_esfera = 'E'
                        else:
                            co_esfera = 'M'
                        filters_siconfi = '?an_exercicio=' + an_exercicio + '&in_periodicidade=' + periodicidade + '&nr_periodo=' + nr_periodo + '&co_tipo_demonstrativo=' + tipo_demonstrativo + '&no_anexo=' + no_anexo + '&co_esfera=' + co_esfera + '&co_poder=' + co_poder + '&id_ente=' + str(id_ente)
                        time.sleep(tempo)
                        df_aux = CarregarDados(ConectarAPI(host_siconfi, dataset_siconfi[0], filters_siconfi), 'siconfi')
                        if df.empty:
                            df = df_aux
                        else:
                            df = pd.concat([df, df_aux], ignore_index=True)
            n += 1
            progresso = n/ciclo*100
            print(msg_processamento_progresso.format(progresso))
    return df

def ProcessarDCA():
    """Função que processa o dataset dca"""
    n = 0
    ciclo = len(exercicios) * len(entes)
    df = pd.DataFrame()
    for an_exercicio in exercicios:
        for id_ente in entes:
            if len(anexos) == 7:
                df_aux = pd.DataFrame()
                filters_siconfi = '?an_exercicio=' + an_exercicio + '&id_ente=' + str(id_ente)
                df_aux = CarregarDados(ConectarAPI(host_siconfi, dataset_siconfi[0], filters_siconfi), 'siconfi')
                if df.empty:
                    df = df_aux
                else:
                    df = pd.concat([df, df_aux], ignore_index=True)
            else:
                for no_anexo in anexos:
                    df_aux = pd.DataFrame()
                    filters_siconfi = '?an_exercicio=' + an_exercicio + '&no_anexo=' + no_anexo + '&id_ente=' + str(id_ente)
                    time.sleep(tempo)
                    df_aux = CarregarDados(ConectarAPI(host_siconfi, dataset_siconfi[0], filters_siconfi), 'siconfi')
                    if df.empty:
                        df = df_aux
                    else:
                        df = pd.concat([df, df_aux], ignore_index=True)
            n += 1
            progresso = n/ciclo*100
            print(msg_processamento_progresso.format(progresso))
    return df

def ProcessarMSC():
    """Função que processa os datasets msc_patrimonial, msc_orcamentaria e msc_controle"""
    n = 0
    ciclo = len(exercicios) * len(periodos) * len(entes)
    df = pd.DataFrame()
    for an_exercicio in exercicios:
        for nr_periodo in periodos:
            for id_ente in entes:
                for no_anexo in anexos:
                    for classe in classe_conta:
                        for id_tv in tipo_valor:
                            df_aux = pd.DataFrame()
                            filters_siconfi = '?id_ente=' + str(id_ente) + '&an_referencia=' + an_exercicio + '&me_referencia=' + nr_periodo + '&co_tipo_matriz=' + no_anexo + '&classe_conta=' + classe + '&id_tv=' + id_tv
                            time.sleep(tempo)
                            df_aux = CarregarDados(ConectarAPI(host_siconfi, dataset_siconfi[0], filters_siconfi), 'siconfi')
                            if df.empty:
                                df = df_aux
                            else:
                                df = pd.concat([df, df_aux], ignore_index=True)
                n += 1
                progresso = n/ciclo*100
                print(msg_processamento_progresso.format(progresso))
    return df

def GerarArquivo(df, nome_arquivo):
    """Função que gera um arquivo XLSX com os dados do anexo de um determinado relatório"""
    df.to_excel(nome_arquivo)
    print(msg_geracao_arquivo.format(nome_arquivo))

In [ ]:
def ExibirMenuHorizontal(menu):
    """Função que exibe um menu de opções na direção horizontal"""
    #print("\n")
    print(menu[0])
    print(menu[1:])

def ExibirMenuVertical(menu):
    """Função que exibe um menu de opções na direção vertical"""
    #print("\n")
    for n in menu:
        print(n)

## Definição de Configurações

In [ ]:
# APIs

sistema = "Serviço de Dados Abertos do SICONFI"

host_siconfi = "http://apidatalake.tesouro.gov.br/ords/siconfi/tt/"
dataset_siconfi = ""
filters_siconfi = ""

host_ibge = "http://servicodados.ibge.gov.br/api/v1/"
dataset_ibge_estados = "localidades/estados"
dataset_ibge_munic = "localidades/estados/{UF}/municipios"
filters_ibge_estados = "?view=nivelado"
filters_ibge_munic = "?view=nivelado"

# Menus

menu_inicial = ['TIPO DE CONSULTA', '1 Estados', '2 Estado e seus Municípios', 'S Sair']
menu_principal = ['CONSULTAS', '1 Anexos', '2 Entes', '3 Extrato de Entregas', '4 RREO', '5 RGF', '6 DCA', '7 MSC-Patrimonial', '8 MSC-Orçamentária', '9 MSC-Controle', 'S Sair']
menu_estado = ['ESTADO']
menu_poder = ['PODER', 'T = Todos', '1 Legislativo', '2 Executivo ', '3 Judiciário', '4 Ministério Público', '5 Defensoria Pública']
menu_exercicio = ['EXERCÍCIO (a partir de 2015)']

menu_rreo_periodo = ['PERÍODO DO RREO', 'T Todos', '1 = 1º Bim', '2 = 2º Bim', '3 = 3º Bim', '4 = 4º Bim', '5 = 5º Bim', '6 = 6º Bim']
#menu_rreo_tipo_demonstrativo = ['TIPO DE DEMONSTRATIVO', '1 RREO', '2 RREO Simplificado']
menu_rreo_anexo = ['ANEXO DO RREO', 'T Todos', '1 Balanço Orçamentário', '2 Despesas por Função e Subfunção', '3 Receita Corrente Líquida', '4 Receitas e Despesas Previdenciárias', '4G Anexo 04 - RGPS', '4P Anexo 04 - RPPS', '4.0 Anexo 04.0 - RGPS', '4.1 Anexo 04.1', '4.2 Anexo 04.2', '4.3 Anexo 04.3 - RGPS', '5 RREO-Anexo 05', '6 Resultados Primário e Nominal', '7 Restos a Pagar por Poder e Órgão', '9 Receitas de Operações de Crédito e Despesas de Capital', '10G Projeção Atuarial RGPS', '10P Projeção Atuarial RPPS', '11 Receita de Alienação de Ativos e Aplicação dos Recursos', '13 Parcerias Público-Privadas', '14 RREO Simplificado']

menu_rgf_periodo = ['PERÍODO DO RGF', 'T Todos', '1 = 1º Quad', '2 = 2º Quad', '3 = 3º Quad']
#menu_rgf_periodicidade = ['PERIODICIDADE', 'Q', 'S']
#menu_rgf_tipo_demonstrativo = ['TIPO DE DEMONSTRATIVO', '1 RGF', '2 RGF Simplificado']
menu_rgf_anexo = ['ANEXO DO RGF', 'T Todos', '1 Despesa com Pessoal', '2 Dívida Consolidada Líquida', '3 Garantias e Contragarantias de Valores', '4 Operações de Crédito', '5 Disponibilidade de Caixa e dos Restos a Pagar', '6 RGF Simplificado']

menu_dca_anexo = ['ANEXO DCA', 'T Todos', '1 Balanço Patrimonial', '2 Receitas Orçamentárias', '3 Despesas Orçamentárias', '4 Despesas por Função', '5 Execução de Restos a Pagar', '6 Execução de Restos a Pagar por Função', '7 Variações Patrimoniais']

menu_msc_periodo = ['PERÍODO DA MSC', 'T Todos', '1 Janeiro', '2 Fevereiro', '3 Março', '4 Abril', '5 Maio', '6 Junho', '7 Julho', '8 Agosto', '9 Setembro', '10 Outubro', '11 Novembro', '12 Dezembro', '13 Encerramento']
menu_msc_tipo_matriz = ['TIPO DE MATRIZ', 'T Todos', '1 Matriz Agregada', '2 Matriz Encerramento']
menu_msc_classe_conta_patrimonial = ['CLASSE DA CONTA', 'T Todos', '1', '2', '3', '4']
menu_msc_classe_conta_orcamentaria = ['CLASSE DA CONTA', 'T Todos', '5', '6']
menu_msc_classe_conta_controle = ['CLASSE DA CONTA', 'T Todos', '7', '8']
menu_msc_tipo_valor = ['TIPO DE VALOR', 'T Todos', '1 Saldo inicial', '2 Movimento', '3 Saldo final']

# Mensagens

msg_execucao_inicio = "Informe uma opção: "
msg_sigla_estado = "Informe a sigla do Estado ou S para sair: "
msg_exercicio_inicio = "Informe o exercício de início: "
msg_exercicio_fim = "Informe o exercício de fim: "
msg_periodo = "Informe o período de referência ou S para sair: "
msg_codigo_poder = "Informe o código do poder ou S para sair: "

msg_rreo_anexo = "Informe o código do Anexo do RREO ou S para sair: "
msg_rgf_anexo = "Informe o código do Anexo do RGF ou S para sair: "
msg_dca_anexo = "Informe o código do Anexo da DCA ou S para sair: "

msg_msc_tipo_matriz = "Informe o código do Tipo da Matriz ou S para sair: "
msg_msc_classe_conta = "Informe o código do Classe da Conta ou S para sair: "
msg_msc_tipo_valor = "Informe o código do Tipo Valor ou S para sair: "

msg_conexao = "Situação da conexão com a URL {} :\n{} - {}"
msg_processamento_inicio = "Processando..."
msg_processamento_progresso = "{:.2f}%"
msg_processamento_interrupcao = "Processamento interrompido. Os parâmetros informados demandarão um longo tempo para processamento. Reduza a quantidade entes, exercícios, períodos e/ou anexos."
msg_processamento_fim = "Processamento concluído com sucesso."
msg_processamento_fim_tempo = "Processamento concluído com sucesso em {:.2f} minutos."
msg_geracao_arquivo = "Arquivo {} gerado com sucesso."
msg_execucao_fim = "Obrigado e volte sempre!"

# Outros

quebra_linha = "\n"
i = 1000
tempo = 0.5

# Principal

In [ ]:
print(sistema)
while True:
    print(quebra_linha)
    ExibirMenuVertical(menu_inicial)
    tipo_consulta = str.upper(input(msg_execucao_inicio))
    if tipo_consulta.isnumeric() and int(tipo_consulta) in range(1,3):
        print(quebra_linha)
        ExibirMenuVertical(menu_principal)
        consulta = str.upper(input(msg_execucao_inicio))
        if consulta.isnumeric() and int(consulta) in range(1,10):
            dataset_siconfi = DefinirDataset(consulta)
            
            # Parametrização
            if int(consulta) in range(3,10):
                print(quebra_linha)
                entes = DefinirEntes(tipo_consulta)
                exercicios = DefinirExercicios()
                if consulta == '4':
                    periodicidade = 'B'
                    periodos = DefinirPeriodos(menu_rreo_periodo, periodicidade)
                    tipo_demonstrativo = 'RREO'
                    anexos = DefinirAnexoRREO()
                elif consulta == '5':
                    periodicidade = 'Q'
                    periodos = DefinirPeriodos(menu_rgf_periodo, periodicidade)
                    tipo_demonstrativo = 'RGF'
                    anexos = DefinirAnexoRGF()
                    poderes = DefinirPoderes()
                elif consulta == '6':
                    anexos = DefinirAnexoDCA()
                elif int(consulta) in range(7,10):
                    periodicidade = 'M'
                    periodos = DefinirPeriodos(menu_msc_periodo, periodicidade)
                    anexos = DefinirTipoMatriz()
                    classe_conta = DefinirClasseConta(dataset_siconfi[0])
                    tipo_valor = DefinirTipoValor()
            
            # Processamento
            print(quebra_linha)
            print(msg_processamento_inicio)
            df_arquivo = pd.DataFrame()
            inicio = timeit.default_timer()
            if int(consulta) in range(1,3):
                df = CarregarDados(ConectarAPI(host_siconfi, dataset_siconfi[0], filters_siconfi), 'siconfi')
                df_arquivo = df
            elif consulta == '3':
                df_entregas = ProcessarEntregas()
                df_arquivo = df_entregas
            elif consulta == '4' and (len(exercicios) * len(periodos) * len(entes) * len(anexos)) < i:
                df_rreo = ProcessarRREO()
                df_arquivo = df_rreo
            elif consulta == '5' and (len(exercicios) * len(periodos) * len(entes) * len(anexos) * len(poderes)) < i:
                df_rgf = ProcessarRGF()
                df_arquivo = df_rgf
            elif consulta == '6' and (len(exercicios) * len(entes) * len(anexos)) < i:
                df_dca = ProcessarDCA()
                df_arquivo = df_dca
            elif int(consulta) in range(7,10) and (len(exercicios) * len(periodos) * len(entes) * len(anexos) * len(classe_conta) * len(tipo_valor)) < i:
                df_msc = ProcessarMSC()
                df_arquivo = df_msc
            fim = timeit.default_timer()
            if df_arquivo.empty:
                print(msg_processamento_interrupcao)
            else:
                print(msg_processamento_fim_tempo.format((fim - inicio)/60))
                GerarArquivo(df_arquivo, dataset_siconfi[1])
                df_arquivo = pd.DataFrame()
    else:
        print(quebra_linha)
        print(msg_execucao_fim)
        break